In [1]:
import os
import tempfile

import matplotlib.pyplot as plt

from IPython.display import clear_output
from ipywidgets import Label, VBox

In [2]:
from cicliminds_lib.query.files import get_datasets

In [3]:
from cicliminds.widgets.filter import FilterWidget
from cicliminds.widgets.filtered import FilteredWidget
from cicliminds.widgets.staging import StagingWidget
from cicliminds.widgets.staged import StagedWidget
from cicliminds.widgets.block import BlockWidget

from cicliminds.interface import expand_state_into_queries
from cicliminds.backend import write_dataset_by_query
from cicliminds.backend import plot_by_query

In [4]:
plt.rcParams["figure.figsize"] = (12,8)
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 8
plt.rcParams["xtick.major.width"] = 1.6
plt.rcParams["xtick.minor.width"] = 0.8
plt.rcParams["xtick.minor.size"] = 4
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.width"] = 1.6
plt.rcParams["ytick.minor.width"] = 0.8
plt.rcParams["ytick.major.size"] = 8
plt.rcParams["ytick.minor.size"] = 4
plt.rcParams["font.size"] = 16
plt.rcParams["lines.linewidth"] = 3
plt.rcParams["lines.markersize"] = 5
plt.rcParams["savefig.dpi"] = 300/2.4
plt.rcParams["savefig.transparent"] = False
plt.rcParams["savefig.facecolor"] = "white"

In [5]:
DATA_DIR = os.environ["DATA_DIR"]
display(DATA_DIR)

'/home/viktoana/projects/cicero/data/Climdex_base1981-2010'

In [6]:
DATASETS = get_datasets(DATA_DIR)

In [7]:
def update_filters(objs, change):
    filters_widget = objs[0]
    global filtered_widget
    filtered_dataset = filters_widget.get_filtered_dataset()
    filters_widget.update_state_from_dataset(filtered_dataset)
    if filtered_dataset.shape[0] > 200:
        return
    filtered_widget.update_state_from_dataset(filtered_dataset)

In [8]:
def stage_action(objs, change):
    staging_widget = objs[0]
    global filtered_widget
    global staged_widget
    to_agg = filtered_widget.get_selected_dataset()
    agg_params = staging_widget.get_state()
    queries_to_add = list(expand_state_into_queries(to_agg, agg_params))
    staged_widget.add_blocks_from_queries(queries_to_add)

In [9]:
def _is_rebuild_one_action(objs, change):
    block_widget = None
    for obj in objs:
        if isinstance(obj, BlockWidget):
            block_widget = obj
            break
    else:
        return None
    if block_widget.state["rebuild_button"] is change:
        return block_widget
    return None

def rebuild_one_block_action(objs, change):
    block_widget = _is_rebuild_one_action(objs, change)
    if block_widget is None:
        return
    cfg = block_widget.get_query()
    fig, ax = plt.subplots()
    with tempfile.NamedTemporaryFile("r") as dataset:
        write_dataset_by_query(DATASETS, cfg, dataset.name)
        plot_by_query(ax, dataset.name, cfg)
    plt.close()
    with block_widget.capture_output():
        clear_output()
        display(fig)

In [11]:
filter_widget = FilterWidget(DATASETS)
filter_widget.observe(update_filters)
filtered_widget = FilteredWidget()
staging_widget = StagingWidget()
staging_widget.observe(stage_action)
staged_widget = StagedWidget()
staged_widget.observe(rebuild_one_block_action)


app = VBox([filter_widget.render(),
            filtered_widget.render(),
            staging_widget.render(),
            staged_widget.render()])

display(app)
filter_widget.reset_filters()